In [3]:
import pandas as pd
import numpy as np
import utils.geohash_utils as ghu
#visualize
import folium
from branca.colormap import linear

In [ ]:
df = pd.read_feather('/root/autodl-tmp/data_processed/wp.feather')

In [ ]:
precision_max = 5
bounds = (53.9884, 9.7614, 56.4261, 15.1117)
tree = ghu.QuadTree(bounds, precision_max)
tree.init_subgrids(tree.root)
tree.init_ship_dic_df(df)
tree.init_ship_cnt()
tree.filter_nodes(1)
leaf_nodes = tree.get_leaf_nodes()
leaf_nodes = ghu.assign_label(leaf_nodes)
filtered_grids_map = ghu.visualize_node_list(df, leaf_nodes, color='red')
waypoints = ghu.leaf_nodes2waypoints(leaf_nodes)
convex_hulls = ghu.get_convex_hulls(waypoints)
convex_hulls.to_file('convex_hull.geojson', driver='GeoJSON')
df = pd.read_feather('*')
adjacency_matrix = ghu.get_adjacency_matrix(convex_hulls, df)
edges_list, adjacency_matrix = ghu.get_edges_list(convex_hulls, adjacency_matrix, flow_threshold=2, dis_threshold=24)

In [ ]:
edge_list = []
LAT_MAX, LAT_MIN, LON_MAX, LON_MIN = (53.9884, 9.7614, 56.4261, 15.1117)
import folium

m = folium.Map(location=[(LAT_MAX + LAT_MIN) / 2, (LON_MAX + LON_MIN) / 2], zoom_start=8, tiles='OpenStreetMap')
connection_cnt = np.zeros((waypoints.shape[0],))
for i in range(waypoints.shape[0]):

    folium.Circle(
        location=[waypoints[i][0], waypoints[i][1]],
        radius=5,
        color='black',
        fill=True,
        fill_opacity=0.2
    ).add_to(m)

for index, row in gdf.iterrows():
    if isinstance(row['geometry'], Polygon):
        hull = folium.GeoJson(row['geometry'].__geo_interface__, name='convex_hull')
        Popup('wp' + str(row['label'])).add_to(hull)
        hull.add_to(m)


folium.PolyLine(
    locations=edge_list,
    color="black",
    weight=5,
    opacity=0.7,
).add_to(m)
m